In [12]:
import json, folium, warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

In [13]:
park = pd.read_csv('../04. 지도시각화/data/경기도 도시공원정보표준데이터.csv', encoding='euc-kr')
park = park[['공원명','소재지지번주소','공원면적']]
park.dropna(how='any', inplace=True)
park.head()

,공원명,소재지지번주소,공원면적
0,녹색소공원,서울특별시 용산구 효창동 287-6,1569.9
23,강변공원,서울특별시 용산구 이촌동 408,1766.0
24,솔밭어린이공원,서울특별시 용산구 한강로1가 30-3,2090.0
25,신창동어린이공원,서울특별시 용산구 신창동 100,257.0
26,서빙고공원,서울특별시 용산구 용산동5가 24-2,11879.0


In [14]:
# 경기도 데이터만 추출하기
park = park[park.소재지지번주소.str.contains('경기도')]
park.head()

,공원명,소재지지번주소,공원면적
212,꿈나무공원,경기도 이천시 중리동 15,2526.0
213,남천공원,경기도 이천시 중리동 210,3830.0
214,향교공원,경기도 이천시 창전동 408,1757.0
215,개나리공원,경기도 이천시 창전동 414-13,1650.0
216,달맞이공원,경기도 이천시 창전동 423,1650.0


In [15]:
# 3) 소재지지번주소에서 시군구(예, 이천시, 양평군, 수원시 장안구)를 추출해서   시군구 컬럼을 만드세요.
park['시군구'] = park.소재지지번주소.apply(lambda x: 
                    x.split()[1] if x.split()[2][-1] != '구' else ' '.join(x.split()[1:3]))
park

,공원명,소재지지번주소,공원면적,시군구
212,꿈나무공원,경기도 이천시 중리동 15,2526.0,이천시
213,남천공원,경기도 이천시 중리동 210,3830.0,이천시
214,향교공원,경기도 이천시 창전동 408,1757.0,이천시
215,개나리공원,경기도 이천시 창전동 414-13,1650.0,이천시
216,달맞이공원,경기도 이천시 창전동 423,1650.0,이천시
...,...,...,...,...
17025,제90호 어린이공원(풀향기),경기도 용인시 기흥구 중동 1083,1500.0,용인시 기흥구
17258,청석공원,경기도 광주시 경안동 242,193530.0,광주시
17259,경안근린공원,경기도 광주시 경안동 168-1,83095.0,광주시
17260,삼리어린이공원,경기도 광주시 곤지암읍 삼리 131-1,1493.0,광주시


In [16]:
#1. 4) 시군구 별로 공원면적의 합을 구하세요.
park= park.groupby('시군구')['공원면적'].sum().reset_index()
park.head()

,시군구,공원면적
0,가평군,511674.5
1,고양시 덕양구,3287291.4
2,고양시 일산동구,2722980.0
3,고양시 일산서구,922486.9
4,과천시,315666.0


In [17]:
#1. 1) 행정구역구분명, 행정구역명, 총 인구수 컬럼만 선택해서 데이터프레임을   만드세요.
pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황 (1).csv', encoding='euc-kr')
pop =pop[['행정구역구분명','행정구역명','총 인구수']]
pop.head()

,행정구역구분명,행정구역명,총 인구수
0,도,경기도,13589432
1,시군,경기도 가평군,62197
2,읍면동,경기도 가평군 가평읍,19555
3,읍면동,경기도 가평군 북면,3815
4,읍면동,경기도 가평군 상면,5669


In [18]:
#1. 2) 경기도와 읍면동 인구를 제외한 시군과 구의 데이터만 추출하세요.
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()


,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
8,시군,경기도 고양시,1075202
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738


In [19]:
#1. 3) 수원시, 용인시 등 구가 있는 시는 시 데이터를 제거하세요.
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775


In [20]:
1. #4) 시군구 컬럼을 만드세요.
my_list = []
for name in pop.행정구역명:
    if name.find('구') >=0:
        my_list.append(''.join(name.split()[1:3]))
    else:
        my_list.append(''.join(name.split()[1]))
pop['시군구']=my_list
pop.head()

,행정구역구분명,행정구역명,총 인구수,시군구
1,시군,경기도 가평군,62197,가평군
9,구,경기도 고양시 덕양구,487874,고양시덕양구
31,구,경기도 고양시 일산동구,296590,고양시일산동구
44,구,경기도 고양시 일산서구,290738,고양시일산서구
56,시군,경기도 과천시,77775,과천시


In [33]:
#1, 2번에서 구한 데이터프레임을 합성한 후, 인당 공원면적을 구하세요.
df = park.pivot_table('공원면적', '시군구', aggfunc='sum')
df.head()

,공원면적
시군구,
가평군,511674.5
고양시 덕양구,3287291.4
고양시 일산동구,2722980.0
고양시 일산서구,922486.9
과천시,315666.0


In [34]:
#1, 2번에서 구한 데이터프레임을 합성한 후, 인당 공원면적을 구하세요.
df = df.merge(pop, left_index=True, right_on='시군구')
df['인당 공원면적'] = (df.공원면적 / df['총 인구수']).round(1)
df.head()

,공원면적,행정구역구분명,행정구역명,총 인구수,시군구,인당 공원면적
1,511674.5,시군,경기도 가평군,62197,가평군,8.2
56,315666.0,시군,경기도 과천시,77775,과천시,4.1
63,3397531.0,시군,경기도 광명시,288182,광명시,11.8
83,786484.6,시군,경기도 광주시,391704,광주시,2.0
100,912756.3,시군,경기도 구리시,188876,구리시,4.8
